In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from shuler_lab_to_nwb.behavior_datainterface import ShulerBehaviorInterface
from pathlib import Path
from datetime import datetime
from dateutil import tz

In [ ]:
trials_file = str(Path("ES029_2022-09-14_bot72_0_g0/data_2022-09-14_12-13-31.csv"))
reference_ts_file = str(Path("ES029_2022-09-14_bot72_0_g0/ES029_2022-09-14_bot72_0_g0_tcat.imec0.ap.SY_384_6_0.txt").resolve())

data_interface = ShulerBehaviorInterface(
    trials_file_path=trials_file,
    reference_timestamps_file_path=reference_ts_file
)

In [ ]:
metadata = data_interface.get_metadata()
session_start_time = datetime(2020, 1, 1, 12, 30, 0, tzinfo=tz.gettz("US/Pacific"))
metadata['NWBFile'].update(session_start_time=session_start_time)
metadata

In [ ]:
data_interface.run_conversion(
    nwbfile_path="quicktest.nwb",
    metadata=metadata,
    overwrite=True
)

In [ ]:
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget

io = NWBHDF5IO('quicktest.nwb', mode='r')
nwbfile = io.read()

nwb2widget(nwbfile)